<a href="https://colab.research.google.com/github/octopus2023-inc/gensphere/blob/main/examples/agentic_workflows_from_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic workflows from prompts

This notebook will demonstrate how to create agentic workflows from high-level user prompts. GenSphere will process the prompt and create an execution graph of LLM API calls and function calls.   It will also attach tools for function calling and schemas for structured outputs along the way.

**Important:** The project uses openAI's o1-preview to generate the agentic workflow. o1-preview is available only to certain tiers, please check openAI documentation [here](https://help.openai.com/en/articles/9824962-openai-o1-preview-and-o1-mini-usage-limits-on-chatgpt-and-the-api)

Start by installing gensphere and importing modules

In [2]:
!pip install gensphere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.1/450.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49

In [3]:
from gensphere.hub import Hub
from gensphere.genflow import GenFlow
from gensphere.visualizer import Visualizer
import os
import logging
logging.getLogger('composio').setLevel(logging.WARNING)

In [4]:
os.environ['OPENAI_API_KEY']="PLACE-YOUR-OPENAI-API-KEY-HERE" #Notice that this example will use o1-preview, which is not available to all user Tiers.

Now, we will pull from the Hub a GenSphere project, which takes as input a `task.txt` file and outputs a GenSphere project that accomplishes that task.The yml, functions and schemas file are saved locally after the flow execution completes as `text_to_workflow_result_yaml.yaml`, `text_to_workflow_result_functions.py`, `text_to_workflow_result_schemas.py` respectively.

In [10]:
hub=Hub()
hub.pull(push_id='ddd4bfda-a33c-48e7-810c-4a1768973b9a',
         yaml_filename='yaml_file.yaml',
         functions_filename='gensphere_functions.py',
         schema_filename='gensphere_schemas.py',
         save_to_disk=True)

{'yaml_file.yaml': '# text-to-scraper.yaml\r\n\r\nnodes:\r\n  - name: get_gensphere_docs\r\n    type: function_call\r\n    function: get_gensphere_docs_function\r\n    outputs:\r\n      - gensphere_docs\r\n\r\n  - name: read_task\r\n    type: function_call\r\n    function: read_file_as_string\r\n    params:\r\n      file_path: \'task.txt\'\r\n    outputs:\r\n      - task\r\n\r\n#  - name: decompose_task_steps\r\n#    type: llm_service\r\n#    service: openai\r\n#    model: "gpt-4o-2024-08-06"\r\n#    params:\r\n#      prompt: |\r\n#        Consider the following task that needs to be done:\r\n#\r\n#        {{ read_task.task }}\r\n#\r\n#        To accomplish this task successfully, a set of steps, in sequence or not, needs to be executed. \r\n#        Describe in very granular detail how an expert human being would work to accomplish this task.\r\n#        When I say very granular, I mean it.\r\n#        Break into small little steps, as if this expert was explaining to a total beginner

The files `yaml_file.yaml`, `gensphere_functions.py`, `gensphere_schemas.py` have been pulled from the hub saved locally.

You can visualize nodes in this GenSphere project with the Visualizer class, but we won't do it for now.

Now, let's execute the project by defining a task.

In [11]:
task = """Your task is to generate script for 10 YouTube videos, about 5 minutes long each.
Our aim is to generate content for YouTube in an ethical way, while also ensuring we will go viral.
You should discover which are the topics with the highest chance of going viral today by searching the web.
Divide this search into multiple granular steps to get the best out of it. You can use Tavily and Firecrawl_scrape
to search the web and scrape URL contents, respectively. Then you should think about how to present these topics in order to make the video go viral.
Your script should contain detailed text (which will be passed to a text-to-speech model for voiceover),
as well as visual elements which will be passed to as prompts to image AI models like MidJourney.
You have full autonomy to create highly viral videos following the guidelines above.
Be creative and make sure you have a winning strategy."""


with open("task.txt", "w") as text_file:
    text_file.write(task)

flow=GenFlow('yaml_file.yaml','gensphere_functions.py','gensphere_schemas.py')
flow.parse_yaml()
flow.run()

INFO:gensphere.genflow:yaml file /content/yaml_file.yaml passed all consistency checks
INFO:gensphere.genflow:Execution order: ['get_gensphere_docs', 'read_task', 'create_gensphere_files', 'qa_gensphere', 'extract_gensphere_files', 'postprocess_gensphere_files']
INFO:gensphere.genflow.get_gensphere_docs:Executing node 'get_gensphere_docs'
INFO:gensphere.genflow.read_task:Executing node 'read_task'
INFO:gensphere.genflow.create_gensphere_files:Executing node 'create_gensphere_files'
INFO:gensphere.genflow.qa_gensphere:Executing node 'qa_gensphere'
INFO:gensphere.genflow.extract_gensphere_files:Executing node 'extract_gensphere_files'
INFO:gensphere.genflow.postprocess_gensphere_files:Executing node 'postprocess_gensphere_files'


Your agentic workflow was created, and its associated files were saved locally as `text_to_workflow_result_yaml.yaml`, `text_to_workflow_result_functions.py`, `text_to_workflow_result_schemas.py`. You can visualize your project with the Visualizer class:

In [12]:
Visualizer('text_to_workflow_result_yaml.yaml',
           'text_to_workflow_result_functions.py',
           'text_to_workflow_result_schemas.py',
           address='127.0.0.1', port=8050).start_visualization()

INFO:gensphere.graph_builder:Total elements generated: 24


<IPython.core.display.Javascript object>

Now that our agentic workflow is ready, we can execute it. We will need to add a few more api keys.

In [8]:
os.environ['COMPOSIO_API_KEY']="place-your-api-key" #if you don't have one, visit composio.dev
os.environ['FIRECRAWL_API_KEY']="place-your-api-key" # if you don't have one, visit firecrawl.dev
os.environ['TAVILY_API_KEY']="place-your-api-key" # if you don't have one, visit tavily

In [ ]:
!composio add firecrawl #for this project, we will be using firecrawl. Get an API key and add it by following the steps here.

In [ ]:
!composio add tavily #for this project, we will be using tavily. Get an API key and add it by following the steps here.

In [13]:
flow=GenFlow('text_to_workflow_result_yaml.yaml',
           'text_to_workflow_result_functions.py',
           'text_to_workflow_result_schemas.py')
flow.parse_yaml()
flow.run()

INFO:composio:Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
INFO:composio:Logging is set to INFO, use `logging_level` argument or `COMPOSIO_LOGGING_LEVEL` change this
INFO:gensphere.genflow:yaml file /content/text_to_workflow_result_yaml.yaml passed all consistency checks
INFO:gensphere.genflow:Execution order: ['get_current_date', 'generate_search_queries', 'search_viral_topics_tavily', 'extract_urls_from_tavily_results', 'scrape_urls_firecrawl', 'process_scraped_content_to_extract_topics', 'convert_processed_topics_to_dict', 'select_top_10_topics', 'extract_titles_from_selected_topics', 'generate_scripts_for_topics', 'generate_visual_prompts', 'compile_final_output']
INFO:gensphere.genflow.get_current_date:Executing node 'get_current_date'
INFO:gensphere.genflow.generate_search_queries:Executing node 'generate_search_queries'
INFO:gensphere.genflow.search_viral_topics_tavily:Executing node 'search_viral_topics_tavily'
INFO:composio:Loggi

You can now access outputs with `flow.outputs` (which returns a dictionary, where keys are node names and values are also dictionaries, with one key for each node output as defined in the yaml file `text_to_workflow_result_yaml`).

In [ ]:
flow.outputs

By looking at your worklflow's graph, you can access final node outputs. As each call to openAI's API may return different results, check the visualization above to understand exactly how your project was created and what was the final node. This will allow you to adapt the code block below accordingly.

In [19]:
#Print final outputs. You should adapt this according to how your project was created. Check the visualization above.
for i in flow.outputs.get('compile_final_output').get('final_output'):
  script=i.get('script')
  visual_prompt=i.get('visual_prompt')
  print(f'script: {script}')
  print(f'visual prompt: {visual_prompt}')

script: **[INTRO: UPBEAT MUSIC PLAYING]**

**[Visual: Dynamic montage of TikTok challenge clips from 2024 with bright colors and flashy transitions. Clips show a mix of dance, comedy, and creative challenges.]**

**Narrator (VOICEOVER):**  
Welcome back to [Channel Name]! Today, we're diving into the latest craze that's taking over the internet: TikTok Challenges 2024! It's a year of innovation, creativity, and of course, pure fun. So, let's hit the ground running and explore these exciting trends.

**[CUT TO HOST ON CAMERA, SMILING AND ENERGETIC]**

**Host:**  
Hey everyone, [Host Name] here! TikTok has been the breeding ground for creativity, and 2024 is no exception. From mind-blowing dances to hilarious pranks, let's explore the challenges that have taken the platform by storm this year! Ready? Let's go!

**[UPBEAT TRANSITION SOUND]**

**[Visual: Title Card: "Challenge #1: The Time Warp Glow Up"]**

**Narrator (VOICEOVER):**  
First up, we have the "Time Warp Glow Up"! This challen